# Segmenting and Clustering Neighborhoods in Toronto (Part 3)

## Code from Part 1

In [76]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')
PostalCodeList = []
BoroughList = []
NeighborhoodList = []
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):   
        PostalCodeList.append(cells[0].text[:-1])
        BoroughList.append(cells[1].text[:-1])
        NeighborhoodList.append(cells[2].text[:-1])
toronto_df = pd.DataFrame({"PostalCode": PostalCodeList,
                           "Borough": BoroughList,
                           "Neighborhood": NeighborhoodList})
toronto_df = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df = toronto_df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
for index, row in toronto_df.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [77]:
toronto_df.shape

(103, 3)

In [78]:
import numpy as np

## Code for Part 2

In [79]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [80]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [81]:
toronto_df = toronto_df.merge(coordinates, on="PostalCode", how="left")
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [82]:
test_df = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood",'Latitude','Longitude'])
test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]
for postcode in test_list:
    test_df = test_df.append(toronto_df[toronto_df["PostalCode"]==postcode], ignore_index=True)  
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


## Code for Part 3

### Use geopy library to get the latitude and longitude values of Toronto

In [83]:
from geopy.geocoders import Nominatim
import folium

In [84]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-app")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [85]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'],toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Filter only boroughs that contain the word Toronto

In [86]:
toronto_borough_df = toronto_df[toronto_df.Borough.str.contains('Toronto',case=False)]

In [87]:
# this DataFrame contains Boroughs with only Toronto Values
toronto_borough_df = toronto_borough_df.reset_index(drop=True)

In [88]:
toronto_borough_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [89]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_borough_df['Latitude'], toronto_borough_df['Longitude'],
                                           toronto_borough_df['Borough'], toronto_borough_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto 

###  Define Foursquare Credentials and Version

In [90]:
CLIENT_ID = '0ZHCHPCDSDKTLEMQ5TOOOMLYMVGIH22EHM2SUJEDDKJNVT5T' # your Foursquare ID
CLIENT_SECRET = 'DLJWZJKBITFTDRGURCGG4I5OOJKFSZQSAUTYEZQ2ZE3G0K3K' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
ACCESS_TOKEN = '33NW0WGLSAHLEM011BOST3WKE2WN342TMO02EUHHFTWESBXX'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('Your access token is: '+ ACCESS_TOKEN)

Your credentails:
CLIENT_ID: 0ZHCHPCDSDKTLEMQ5TOOOMLYMVGIH22EHM2SUJEDDKJNVT5T
CLIENT_SECRET:DLJWZJKBITFTDRGURCGG4I5OOJKFSZQSAUTYEZQ2ZE3G0K3K
Your access token is: 33NW0WGLSAHLEM011BOST3WKE2WN342TMO02EUHHFTWESBXX


###  Now, let's get the top 100 venues that are within a radius of 500 meters.

In [93]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_borough_df['Latitude'], toronto_borough_df['Longitude'],
                                                  toronto_borough_df['PostalCode'], toronto_borough_df['Borough'],
                                                  toronto_borough_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&oauth_token={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        ACCESS_TOKEN,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [94]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1805, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Domino's Pizza,43.679058,-79.297382,Pizza Place


###  Let's check how many venues were returned for each PostalCode

In [95]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,9,9,9,9,9,9
M4K,East Toronto,"The Danforth West, Riverdale",42,42,42,42,42,42
M4L,East Toronto,"India Bazaar, The Beaches West",32,32,32,32,32,32
M4M,East Toronto,Studio District,30,30,30,30,30,30
M4N,Central Toronto,Lawrence Park,4,4,4,4,4,4
M4P,Central Toronto,Davisville North,15,15,15,15,15,15
M4R,Central Toronto,North Toronto West,43,43,43,43,43,43
M4S,Central Toronto,Davisville,53,53,53,53,53,53
M4T,Central Toronto,"Moore Park, Summerhill East",4,4,4,4,4,4


### Let's find out how many unique categories can be curated from all the returned venues

In [96]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 257 uniques categories.


In [97]:
venues_df['VenueCategory'].unique()[:50]

array(['Trail', 'Health Food Store', 'Pub', 'Park', 'Pizza Place',
       'Neighborhood', 'Coffee Shop', 'Home Service', 'Asian Restaurant',
       'Cosmetics Shop', 'Greek Restaurant', 'Italian Restaurant',
       'Ice Cream Shop', 'Yoga Studio', 'Brewery',
       'Fruit & Vegetable Store', 'Dessert Shop', 'Restaurant',
       'Juice Bar', 'Bookstore', 'Bubble Tea Shop',
       'Furniture / Home Store', 'Grocery Store', 'Spa', 'Bakery',
       'Caribbean Restaurant', 'Café', 'Indian Restaurant', 'Lounge',
       'Frozen Yogurt Shop', 'American Restaurant', 'Liquor Store', 'Gym',
       'Fish & Chips Shop', 'Fast Food Restaurant', 'Sushi Restaurant',
       'Pet Store', 'Steakhouse', 'Burrito Place', 'Movie Theater',
       'Sandwich Place', 'Rental Car Location', 'Board Shop',
       'Light Rail Station', 'Nightlife Spot', 'Intersection',
       'Mobile Phone Shop', 'Gym / Fitness Center', 'Motorcycle Shop',
       'Food & Drink Shop'], dtype=object)

### One Hot Coding to analyze each area

In [98]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1805, 260)


,PostalCode,Borough,Neighborhoods,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Regroup the data based on PostaCode,Borough and Neighborhoods

In [99]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

In [100]:
toronto_grouped.head()

,PostalCode,Borough,Neighborhoods,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.023810
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,M4M,East Toronto,Studio District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.033333
4,M4N,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


#### Now let's create the new dataframe and display the top 10 venues for each PostalCode.

In [101]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Asian Restaurant,Park,Health Food Store,Pizza Place,Coffee Shop,Home Service,Neighborhood,Trail,Pub,Donut Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Ice Cream Shop,Bookstore,Restaurant,Brewery,Bubble Tea Shop,Café
2,M4L,East Toronto,"India Bazaar, The Beaches West",Park,Pizza Place,Fast Food Restaurant,Sandwich Place,Coffee Shop,Burrito Place,Italian Restaurant,Steakhouse,Fish & Chips Shop,Light Rail Station
3,M4M,East Toronto,Studio District,Café,Bakery,American Restaurant,Coffee Shop,Yoga Studio,Thai Restaurant,Bookstore,Sandwich Place,Brewery,Fish Market
4,M4N,Central Toronto,Lawrence Park,Bus Line,Gym / Fitness Center,Park,Lawyer,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
5,M4P,Central Toronto,Davisville North,Hotel,Pizza Place,Supplement Shop,Gym,Food & Drink Shop,Electronics Store,Dog Run,Department Store,Convenience Store,Construction & Landscaping
6,M4R,Central Toronto,North Toronto West,Clothing Store,Cosmetics Shop,Sporting Goods Shop,Salon / Barbershop,Rental Car Location,Coffee Shop,Spa,Italian Restaurant,Pet Store,Shoe Store
7,M4S,Central Toronto,Davisville,Pizza Place,Spa,Dessert Shop,Sandwich Place,Sushi Restaurant,Thai Restaurant,Italian Restaurant,Café,Toy / Game Store,Coffee Shop
8,M4T,Central Toronto,"Moore Park, Summerhill East",Beer Store,Playground,Park,Tennis Court,Construction & Landscaping,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Pub,Coffee Shop,Light Rail Station,Lawyer,Bank,Aquarium,Frozen Yogurt Shop,Sushi Restaurant,Bagel Shop,Restaurant


### K-Means Clustering
#### set the value of K and define the number of clusters

In [102]:
from sklearn.cluster import KMeans

In [103]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 2, 0, 0, 0, 3, 0])

In [104]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_borough_df.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Asian Restaurant,Park,Health Food Store,Pizza Place,Coffee Shop,Home Service,Neighborhood,Trail,Pub,Donut Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Ice Cream Shop,Bookstore,Restaurant,Brewery,Bubble Tea Shop,Café
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Pizza Place,Fast Food Restaurant,Sandwich Place,Coffee Shop,Burrito Place,Italian Restaurant,Steakhouse,Fish & Chips Shop,Light Rail Station
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Bakery,American Restaurant,Coffee Shop,Yoga Studio,Thai Restaurant,Bookstore,Sandwich Place,Brewery,Fish Market
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Bus Line,Gym / Fitness Center,Park,Lawyer,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio


In [105]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Asian Restaurant,Park,Health Food Store,Pizza Place,Coffee Shop,Home Service,Neighborhood,Trail,Pub,Donut Shop
20,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,0,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Seafood Restaurant,Deli / Bodega,American Restaurant,Tea Room,Italian Restaurant
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0,Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,Breakfast Spot
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Sandwich Place,Café,Coffee Shop,Grocery Store,Park,Burger Joint,Rental Car Location,Pub,Cheese Shop,Pizza Place
25,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,0,Bookstore,Restaurant,Bakery,Japanese Restaurant,Theater,Sandwich Place,Beer Bar,Bar,French Restaurant,Sushi Restaurant
26,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,0,Café,Chinese Restaurant,Mexican Restaurant,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Caribbean Restaurant,Bakery,Ice Cream Shop,Coffee Shop
27,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,0,Airport Service,Rental Car Location,Airport Lounge,Airport Terminal,Harbor / Marina,Boat or Ferry,Coffee Shop,Plane,Sculpture Garden,Bar
28,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,0,Coffee Shop,Restaurant,Cocktail Bar,Seafood Restaurant,Café,Beer Bar,Hotel,Japanese Restaurant,Italian Restaurant,Bakery
29,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0,Coffee Shop,Café,Hotel,Japanese Restaurant,Gym,Restaurant,Salad Place,Steakhouse,Asian Restaurant,Seafood Restaurant
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Rental Car Location,Park,Coffee Shop,Italian Restaurant,Business Service,Bank,Sushi Restaurant,Japanese Restaurant


In [106]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [107]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Lets examine the Clusters

#### Cluster 1

In [108]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Asian Restaurant,Park,Health Food Store,Pizza Place,Coffee Shop,Home Service,Neighborhood,Trail,Pub,Donut Shop
20,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Seafood Restaurant,Deli / Bodega,American Restaurant,Tea Room,Italian Restaurant
21,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,Breakfast Spot
24,Central Toronto,0,Sandwich Place,Café,Coffee Shop,Grocery Store,Park,Burger Joint,Rental Car Location,Pub,Cheese Shop,Pizza Place
25,Downtown Toronto,0,Bookstore,Restaurant,Bakery,Japanese Restaurant,Theater,Sandwich Place,Beer Bar,Bar,French Restaurant,Sushi Restaurant
26,Downtown Toronto,0,Café,Chinese Restaurant,Mexican Restaurant,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Caribbean Restaurant,Bakery,Ice Cream Shop,Coffee Shop
27,Downtown Toronto,0,Airport Service,Rental Car Location,Airport Lounge,Airport Terminal,Harbor / Marina,Boat or Ferry,Coffee Shop,Plane,Sculpture Garden,Bar
28,Downtown Toronto,0,Coffee Shop,Restaurant,Cocktail Bar,Seafood Restaurant,Café,Beer Bar,Hotel,Japanese Restaurant,Italian Restaurant,Bakery
29,Downtown Toronto,0,Coffee Shop,Café,Hotel,Japanese Restaurant,Gym,Restaurant,Salad Place,Steakhouse,Asian Restaurant,Seafood Restaurant
30,Downtown Toronto,0,Grocery Store,Café,Rental Car Location,Park,Coffee Shop,Italian Restaurant,Business Service,Bank,Sushi Restaurant,Japanese Restaurant


#### Cluster 2

In [109]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,1,Bus Line,Jewelry Store,Park,Sushi Restaurant,Home Service,Trail,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant


#### Cluster 3

In [110]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Bus Line,Gym / Fitness Center,Park,Lawyer,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio


#### Cluster 4

In [111]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,3,Beer Store,Playground,Park,Tennis Court,Construction & Landscaping,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
10,Downtown Toronto,3,Park,Trail,Playground,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


#### Cluster 5

In [112]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,4,Home Service,Health & Beauty Service,Pool,Ice Cream Shop,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


## Observations


#### Cluster 1 which has a lot of Coffee Shops, Parks, Hotels and Gyms.  
#### Cluster 2 with the most common venue being the garden but only one Borough.
#### Cluster 3 is also a single Borough with Bus Line as the most commpn Venue.
#### Cluster 4 has two Boroughs with Beer Store and Park as the most commpn Venues.
#### Cluster 5 is also a single Borough with Home Service as the most commpn Venue.

# Thank You!